In [ ]:
EXAMPLE_1 = "../example_1.txt"
EXAMPLE_2 = "../example_2.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    wires = []
    gates = []
    with open(input_file_name, 'r') as f:
        reading_wires = True
        for line in f:
            if line == '\n':
                reading_wires = False
                continue
            if reading_wires:
                wire, value = line.strip().replace('\n', '').split(':')
                value = bool(int(value.strip()))
                wires.append((wire, value))
            else:
                inputs, output = line.strip().replace('\n', '').split('->')
                output = output.strip()
                input0, gate, input1 = [s.strip() for s in inputs.strip().split(' ')]
                gates.append((input0, gate, input1, output))
    return wires, gates

In [ ]:
wires, gates = parse_input(EXAMPLE_1)
print(wires, gates)

In [ ]:
class Gate:
    name: str
    op: str
    inputs: list[bool | None]
    output: bool | None
    children: list[str]

    def __init__(self, op, name):
        self.name = name
        self.output = None
        self.inputs = [None, None]
        self.op = op
        self.children = []

    def add_child(self, name):
        self.children.append(name)

    def set_input(self, value, device):
        for i in range(2):
            if self.inputs[i] is None:
                self.inputs[i] = value
                break
        if None not in self.inputs:
            self.generate_output(device)

    def generate_output(self, device):
        if self.inputs[0] is None or self.inputs[1] is None:
            return
        match self.op:
            case "AND":
                self.output = self.inputs[0] and self.inputs[1]
            case "OR":
                self.output = self.inputs[0] or self.inputs[1]
            case "XOR":
                self.output = self.inputs[0] ^ self.inputs[1]
            case "PASS":
                self.output = self.inputs[0]
        for child in self.children:
            device[child].set_input(self.output, device)

In [ ]:
def build_device(wires, gates):
    DEVICE = {}
    for name, _ in wires:
        new_gate = Gate('PASS', name)
        DEVICE[name] = new_gate

    for input0, op, input1, name in gates:
        if name in DEVICE:
            DEVICE[name].op = op
        else:
            new_gate = Gate(op=op, name=name)
            DEVICE[name] = new_gate
        if input0 in DEVICE:
            DEVICE[input0].add_child(name)
        else:
            DEVICE[input0] = Gate(op=None, name=input0)
            DEVICE[input0].add_child(name)
        if input1 in DEVICE:
            DEVICE[input1].add_child(name)
        else:
            DEVICE[input1] = Gate(op=None, name=input1)
            DEVICE[input1].add_child(name)
    
    return DEVICE

In [ ]:
DEVICE = build_device(wires, gates)

In [ ]:
def start_device(device, wires):
    for name, state in wires:
        device[name].set_input(state, device)
        device[name].set_input(state, device)

In [ ]:
start_device(DEVICE, wires)

In [ ]:
for gate in DEVICE.values():
    print(gate.name, gate.output)

In [ ]:
def get_number(device):
    number = 0
    for name, gate in device.items():
        if name.startswith('z'):
            index = int(name[1:])
            value = int(gate.output)
            number += 2**index * value
    return number

In [ ]:
print(get_number(DEVICE))

In [ ]:
def part_1(input_file_name):
    wires, gates = parse_input(input_file_name)
    DEVICE = build_device(wires, gates)
    start_device(DEVICE, wires)
    result = get_number(DEVICE)
    print(result)

In [ ]:
part_1(EXAMPLE_1)

In [ ]:
part_1(EXAMPLE_2)

In [ ]:
part_1(INPUT)

In [ ]:
wires, gates = parse_input(INPUT)

The device is supposed to be an adder. Let's define how an adder should work, bit by bit and then compare with the device we have in order to find discrepancies.

For each new bit position, let's calculate the expected values for xn&yn xn^yn, zn and cn (carry-over)

In [ ]:
nb_of_wires = 45

def get_gates_for_inputs(input0, input1, gates):
    result = {}
    for (i0, op, i1, output) in gates:
        if sorted([input0, input1]) == sorted([i0, i1]):
            result[op] = output
    return result

def get_gates_for_output(output, gates):
    result = {}
    for (i0, op, i1, o) in gates:
        if o == output:
            result[op] = sorted([i0, i1])
    return result

def swap_gates(gates, gates_to_swap):
    for k, (i0, op, i1, o) in enumerate(gates):
        new_o = None
        for idx, g in enumerate(gates_to_swap):
            if o == g and new_o is None:
                new_o = gates_to_swap[1-idx]
        gates[k] = (i0, op, i1, new_o if new_o is not None else o)

regs_to_swap = []

i = 0

while i < nb_of_wires:
    x = f"x{i:02d}"
    y = f"y{i:02d}"
    z = f"z{i:02d}"

    matching_gates = get_gates_for_inputs(x, y, gates)
    xi_xor_yi = matching_gates["XOR"]
    xi_and_yi = matching_gates["AND"]
    if "OR" in matching_gates:
        print("Error")

    print(f"{x} AND {y} = {xi_and_yi}")
    print(f"{x} XOR {y} = {xi_xor_yi}")
        
    if i == 0:
        # Checking def for z_i
        matching_gates = get_gates_for_inputs(x, y, gates)
        if "XOR" in matching_gates and matching_gates["XOR"] == z:
            # z00 is correctly defined
            print(f"z00 correctly defined: {y} XOR {x} = {z}")
        else:
            print("DISCREPANCY")
        # The value to carry over is x00 & y00
        prev_c = xi_and_yi
        print(f"New co: {prev_c}")
        i += 1
        continue

    print(f"Prev Carry-over = {prev_c}")

    # Checking def for z_i
    matching_gates = get_gates_for_inputs(xi_xor_yi, prev_c, gates)
    if "OR" in matching_gates:
        print("WTF")
        break
    if "XOR" in matching_gates and "AND" in matching_gates:
        should_be_z = matching_gates["XOR"]
        right_reg_for_c = matching_gates["AND"]
        if should_be_z == z:
            # z_i is correctly defined
            print(f"{z} correctly defined: {xi_xor_yi} XOR {prev_c} = {z}")
            # right hand register for c_i
            right_reg_for_c = right_reg_for_c
        elif right_reg_for_c == z:
            print(f"Need to swap {matching_gates["AND"]} with {should_be_z}")
            regs_to_swap.extend([right_reg_for_c, should_be_z])
            swap_gates(gates, [right_reg_for_c, should_be_z])
            continue
            # right_reg_for_c = should_be_z
        else:
            print(f"Missing def for {z}")
            print(f"Need to swap {should_be_z} with {z}")
            regs_to_swap.extend([z, should_be_z])
            swap_gates(gates, [should_be_z, z])
            continue
    else:
        print(f"Missing XOR or AND gate for {xi_xor_yi} and {prev_c}")
        matching_gates = get_gates_for_output(z, gates)
        if "AND" in matching_gates or "OR" in matching_gates:
            print("WTF2")
        if "XOR" in matching_gates:
            swap = False
            if xi_xor_yi == matching_gates["XOR"][0]:
                print(f"Need to swap {matching_gates["XOR"][1]} with {prev_c}")
                gates_to_swap = [prev_c, matching_gates["XOR"][1]]
                # prev_c = matching_gates["XOR"][1]
                swap = True
            elif xi_xor_yi == matching_gates["XOR"][1]:
                print(f"Need to swap {matching_gates["XOR"][0]} with {prev_c}")
                gates_to_swap = [prev_c, matching_gates["XOR"][0]]
                # prev_c = matching_gates["XOR"][0]
                swap = True
            elif prev_c == matching_gates["XOR"][0]:
                print(f"Need to swap {matching_gates["XOR"][1]} with {xi_xor_yi}")
                gates_to_swap = [xi_xor_yi, matching_gates["XOR"][1]]
                # xi_xor_yi = matching_gates["XOR"][1]
                swap = True
            elif prev_c == matching_gates["XOR"][1]:
                print(f"Need to swap {matching_gates["XOR"][0]} with {xi_xor_yi}")
                gates_to_swap = [xi_xor_yi, matching_gates["XOR"][0]]
                # xi_xor_yi = matching_gates["XOR"][0]
                swap = True
            if swap:
                regs_to_swap.extend(gates_to_swap)
                swap_gates(gates, gates_to_swap)
                continue
            else:
                print("Swap failed")
        else:
            print("WTF3")
            break

    # Checking def for c_i
    matching_gates = get_gates_for_inputs(xi_and_yi, right_reg_for_c, gates)
    if "OR" in matching_gates:
        if matching_gates["OR"] == z:
            print(f"Need to swap {matching_gates["OR"]} with {should_be_z}")
            regs_to_swap.extend([matching_gates["OR"], should_be_z])
            swap_gates(gates, [matching_gates["OR"], should_be_z])
            continue
        else:
            prev_c = matching_gates["OR"]
    else:
        print(f"Missing OR gate for {xi_and_yi} and {right_reg_for_c}")
        break
    i += 1

print(",".join(sorted(regs_to_swap)))
# bpf,fdw,hcc,hqc,qcw,z05,z11,z35